In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, KFold
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd
from pylab import rcParams
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from collections import defaultdict
from textwrap import wrap
from transformers import  AdamW, get_linear_schedule_with_warmup, RobertaTokenizer, RobertaModel, AutoTokenizer
# import torch
from transformers import TFRobertaForSequenceClassification
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)
# from torch import nn, optim
# from torch.utils.data import Dataset, DataLoader

# Random seed for reproducibilty
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
# torch.manual_seed(RANDOM_SEED)

# Set GPU
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(device)
print("Num GPUs Available: ", tf.config.list_physical_devices('GPU'))


pdf_src = '../IMDB-Dataset-GoogleTranslate-proccessed-nefnir.csv'

df = pd.read_csv(pdf_src)
df.drop(['Unnamed: 0'], axis=1, inplace=True)

def convert(sentiment):
    return 1 if sentiment == 'positive' else 0

df['sentiment'] = df.sentiment.apply(convert)

# df = df.sample(n=100, random_state=RANDOM_SEED)

df_train, df_val_test = train_test_split(df, test_size=0.3, random_state=RANDOM_SEED)
df_val, df_test = train_test_split(df_val_test, test_size=0.5, random_state=RANDOM_SEED)


print(df_train.shape, df_val.shape, df_test.shape)

MODEL_NAME = 'mideind/IceBERT'

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, truncation=True)
# model = TFRobertaForSequenceClassification.from_pretrained(MODEL_NAME)
tokenizer

class_names = ['negative', 'positive']

MAX_LEN = 512
# Number of iterations 
EPOCHS = 20
# Create train, test and val data loaders
BATCH_SIZE = 4

1 Physical GPUs, 1 Logical GPUs
Num GPUs Available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
(70, 2) (15, 2) (15, 2)


In [4]:
def convert_example_to_feature(review):
    return tokenizer.encode_plus(
        review,
        add_special_tokens=True,
        max_length=MAX_LEN,
        return_token_type_ids=False,
        pad_to_max_length=True,
        return_attention_mask=True
    )

def map_example_to_dict(input_ids, attention_masks, label):
  return {
      "input_ids": input_ids,
      "attention_mask": attention_masks,
  }, label

def encode_examples(ds):
  # prepare list, so that we can build up final TensorFlow dataset from slices.
  input_ids_list = []
  attention_mask_list = []
  label_list = []
  for review, label in ds.to_numpy():
    bert_input = convert_example_to_feature(review)
    input_ids_list.append(bert_input['input_ids'])
    attention_mask_list.append(bert_input['attention_mask'])
    label_list.append([label])
  return tf.data.Dataset.from_tensor_slices((input_ids_list, attention_mask_list, label_list)).map(map_example_to_dict)

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import LearningRateScheduler

def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

lr_scheduler = LearningRateScheduler(scheduler)

early_stopping_monitor = EarlyStopping(
    patience=3,
    min_delta=0.01,
    monitor='val_loss',
    mode='min',
    verbose=1
)

learning_rate = 2e-5
model = TFRobertaForSequenceClassification.from_pretrained(MODEL_NAME)
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=1e-08)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')


ds_train_encoded = encode_examples(df_train).batch(BATCH_SIZE)
ds_val_encoded = encode_examples(df_val).batch(BATCH_SIZE)
ds_test_encoded = encode_examples(df_test).batch(BATCH_SIZE)


model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

model.fit(ds_train_encoded, epochs=EPOCHS, validation_data=ds_val_encoded, callbacks=[lr_scheduler, early_stopping_monitor])

model.evaluate(ds_test_encoded)

model.save_pretrained('icebert_sentiment_model.bin')



Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

Epoch 1/20
18/18 [==============================] - 22s 390ms/step - loss: 0.7345 - accuracy: 0.4714 - val_loss: 0.6709 - val_accuracy: 0.6000 - lr: 2.0000e-05
Epoch 2/20
 6/18 [=========>....................] - ETA: 2s - loss: 0.6093 - accuracy: 0.7917

KeyboardInterrupt: 

In [17]:


test_sentence = "þetta er mjög skemmtileg mynd"

predict_input = tokenizer.encode(test_sentence,truncation=True,padding=True,return_tensors="tf")
tf_output = model.predict(predict_input)[0]
tf_prediction = tf.nn.softmax(tf_output, axis=1)
labels = ['Negative','Positive'] #(0:negative, 1:positive)
print(tf_prediction)
label = tf.argmax(tf_prediction, axis=1)
label = label.numpy()
print(labels[label[0]])


1/1 [==============================] - 0s 36ms/step
tf.Tensor([[0.50161415 0.49838588]], shape=(1, 2), dtype=float32)
Negative
